In [1]:
from google.colab import drive
drive.mount('/content.drive')

Drive already mounted at /content.drive; to attempt to forcibly remount, call drive.mount("/content.drive", force_remount=True).


In [2]:
# import
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [31]:
#2020년 기준으로 진행하였다. 이후 data만 바꿔서 2021년, 2022년 진행.

data = pd.read_csv("/content.drive/MyDrive/data/BDP_PROJECT/2020/data2020.csv")




/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
#필요없는 column들은 삭제
data.drop(['Unnamed: 0','Unnamed: 0.1','대여소번호','대여거치대','반납대여소번호','반납거치대'],axis=1,inplace=True)


이상치 삭제 (임의로 작성)

In [33]:
#이용시간이 0인 데이터들은 삭제.
drop_idx = data[data['이용시간']==0].index
data.drop(drop_idx,inplace = True)


In [34]:
#대여소명과 반납소명이 같으면서 이용시간이 30분 미만인 데이터들은 모두 이용거리가 0이었다 -> 삭제
error_data = data[(data['대여대여소명']==data['반납대여소명'])&(data['이용시간']<20)].index
data.drop(error_data,inplace=True)

In [35]:
#이용시간이 10분 이하이면서 이용거리가 5000m 넘는 것들은 이상치라고 판단하고 삭제한다.
error_data2 = data[(data['이용시간']<10)&(data['이용거리']>5000)].sort_values('이용거리').index
data.drop(error_data2,inplace=True)

In [36]:
data.isnull().sum()


자전거번호      0
대여일시       0
대여대여소명     0
반납일시       0
반납대여소명     0
이용시간       0
이용거리      29
dtype: int64

In [48]:
# 이용거리가 0인 것들 다 NaN으로 바꿔준다.

data['이용거리'] = data['이용거리'].replace(0,np.NaN)

In [49]:
data.isnull().sum()

자전거번호            0
대여일시             0
대여대여소명           0
반납일시             0
반납대여소명           0
이용시간             0
이용거리      12438616
dtype: int64

결측치 채우기
(이용시간이 0인 데이터들)

결측치를 제외한 값들로 학습시킨 뒤, 선형 회귀 모형을 사용하여 값을 추측하고 결측치를 채워넣는다.

In [50]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [51]:
lin_reg = linear_model.LinearRegression()

In [52]:
data

,자전거번호,대여일시,대여대여소명,반납일시,반납대여소명,이용시간,이용거리
0,SPB-36089,2020-07-01 00:00:54,쌍문현대1차아파트 108동 앞,2020-07-01 00:03:13,삼익세라믹아파트교차로,2.0,NaN
1,SPB-11402,2020-07-01 00:00:15,동대문중 교차로,2020-07-01 00:03:23,동대문롯데캐슬아파트 앞,3.0,450.0
2,SPB-36935,2020-07-01 00:01:02,서울숲리버뷰자이APT105동 앞,2020-07-01 00:04:15,응봉역 1번출구,3.0,NaN
3,SPB-35374,2020-07-01 00:00:04,마포역 2번출구 뒤,2020-07-01 00:04:22,용강동 주민센터 앞,4.0,NaN
6,SPB-32502,2020-07-01 00:00:40,도봉구민회관,2020-07-01 00:04:30,신원리베르텔 앞,4.0,NaN
...,...,...,...,...,...,...,...
16220852,SPB-32717,2020.6.16 22:55,이마트 은평점,2020.6.16 23:03,역촌파출소,8.0,NaN
16220853,SPB-15338,2020.6.16 22:46,연가초등학교 옆,2020.6.16 23:03,홍연2교옆,17.0,NaN
16220854,SPB-31982,2020.6.16 22:48,왕십리역 4번 출구 건너편,2020.6.16 23:03,성동세무서 부근,16.0,NaN
16220855,SPB-24981,2020.6.16 22:10,마곡엠밸리10단지 앞,2020.6.16 23:03,마곡엠밸리10단지 앞,52.0,10290.0


In [54]:
#데이터에서 이용거리가 NaN인 데이터들을 삭제한다.
df = data.dropna(axis=0)

In [55]:
df

,자전거번호,대여일시,대여대여소명,반납일시,반납대여소명,이용시간,이용거리
1,SPB-11402,2020-07-01 00:00:15,동대문중 교차로,2020-07-01 00:03:23,동대문롯데캐슬아파트 앞,3.0,450.00
9,SPB-33694,2020-07-01 00:00:17,청담역 13번 출구 앞,2020-07-01 00:05:10,금원빌딩 앞,5.0,492.63
11,SPB-31475,2020-07-01 00:01:56,영풍마드레빌,2020-07-01 00:05:35,금란주차장 앞,4.0,222.39
12,SPB-32198,2020-07-01 00:01:09,으뜸공원,2020-07-01 00:05:50,가로공원공영주차장 1번 출구,5.0,440.87
15,SPB-21915,2020-07-01 00:01:53,앙카라공원 앞,2020-07-01 00:06:47,시범아파트버스정류장 옆,4.0,990.00
...,...,...,...,...,...,...,...
16220845,SPB-12089,2020.6.16 22:10,마곡엠밸리10단지 앞,2020.6.16 23:03,마곡엠밸리10단지 앞,50.0,6830.00
16220847,SPB-23944,2020.6.16 21:56,사당역 7번출구쪽,2020.6.16 23:03,목월공원 앞,64.0,48000.00
16220849,SPB-17079,2020.6.16 22:11,마곡엠밸리10단지 앞,2020.6.16 23:03,마곡엠밸리10단지 앞,51.0,8660.00
16220851,SPB-21493,2020.6.16 22:05,구일역 1번 출입구 앞,2020.6.16 23:03,한진아파트 입구,57.0,11180.00


In [56]:
#그 데이터들로 train, test set을 나눈다.

X = df[['이용시간']]
y = df[['이용거리']]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=1004)

In [57]:
print(X_train.shape)
print(X_test.shape)


(2694616, 1)
(673655, 1)


In [58]:
lin_reg_model = lin_reg.fit(X_train,y_train)

In [59]:
y_pred = lin_reg_model.predict(X_test)
print(r2_score(y_test,y_pred))

#결정계수가 대략 0.40 정도 나온다.

0.401792420422422


In [61]:
# data['이용시간']을 모델에 넣고 all_pred에 담는다.

all_pred = lin_reg_model.predict(data.loc[:,['이용시간']])
all_pred

array([[1030.84000211],
       [1147.93927886],
       [1147.93927886],
       ...,
       [2670.22987663],
       [6885.80383968],
       [4075.42119764]])

In [62]:
# null 값이 발견되면 그 자리에 예측값으로 대체한다.

data['이용거리']= np.where(data['이용거리'].isnull(),
                       pd.Series(all_pred.flatten()),
                       data['이용거리'])

In [63]:
data.isnull().sum()

자전거번호     0
대여일시      0
대여대여소명    0
반납일시      0
반납대여소명    0
이용시간      0
이용거리      0
dtype: int64

In [69]:
data[data['이용시간']==2]

,자전거번호,대여일시,대여대여소명,반납일시,반납대여소명,이용시간,이용거리
0,SPB-36089,2020-07-01 00:00:54,쌍문현대1차아파트 108동 앞,2020-07-01 00:03:13,삼익세라믹아파트교차로,2.0,1030.840002
130,SPB-33809,2020-07-01 00:13:25,신촌역(2호선) 6번출구 옆,2020-07-01 00:15:42,서강대역 2번출구 앞,2.0,283.800000
166,SPB-38967,2020-07-01 00:15:47,SK망원동주유소 건너편,2020-07-01 00:17:38,마포구청역,2.0,1030.840002
262,SPB-32987,2020-07-01 00:21:03,광진청소년수련관,2020-07-01 00:22:36,광나루역 3번 출구,2.0,1030.840002
274,SPB-06929,2020-07-01 00:20:14,선잠단지 앞,2020-07-01 00:23:10,홍익중고 입구,2.0,310.000000
...,...,...,...,...,...,...,...
16219645,SPB-11140,2020.6.16 21:59,홍대입구역 8번출구 앞,2020.6.16 22:46,제2구민체육센타 앞,2.0,570.000000
16220198,SPB-32053,2020.6.16 22:51,월곡역 입구 육교 밑,2020.6.16 22:53,래미안라센트아파트 103동 앞,2.0,1030.840002
16220365,SPB-10708,2020.6.16 22:53,망원1동주민센터,2020.6.16 22:56,마포구청역,2.0,570.000000
16220531,SPB-12736,2020.6.16 22:56,서울신성초등학교,2020.6.16 22:58,동방1교,2.0,300.000000


In [72]:
data.drop(['대여대여소명','반납대여소명','반납일시'],axis=1,inplace=True)

KeyError: ignored

In [74]:
data


,자전거번호,대여일시,이용시간,이용거리
0,SPB-36089,2020-07-01 00:00:54,2.0,1030.840002
1,SPB-11402,2020-07-01 00:00:15,3.0,450.000000
2,SPB-36935,2020-07-01 00:01:02,3.0,1147.939279
3,SPB-35374,2020-07-01 00:00:04,4.0,1265.038556
6,SPB-32502,2020-07-01 00:00:40,4.0,1265.038556
...,...,...,...,...
16220852,SPB-32717,2020.6.16 22:55,8.0,1733.435663
16220853,SPB-15338,2020.6.16 22:46,17.0,2787.329153
16220854,SPB-31982,2020.6.16 22:48,16.0,2670.229877
16220855,SPB-24981,2020.6.16 22:10,52.0,10290.000000


In [82]:
data.to_csv("tmp.csv")